# Extracting Gameflow Information from Shaikh et al.

(Shaikh et al)[https://arxiv.org/abs/2306.02475] published [data](https://github.com/SALT-NLP/codenames/tree/main/data) for training machine learning models on predicting clues and guesses based on demographic information of the players for Codenames Duet games. I downloaded data for the 'clue generation task' and the 'generate_guess_task' to try and extract game metrics. Beforehand, I removed all demographic information from the data that is not relevant to recreating the games.

In [1]:
import pandas

In [2]:
# data downloaded from the 'clue generation task'
cluegiver_data = pandas.read_csv('./baseline_data/cluegiver_shaikh.csv', index_col = 0, delimiter=';')

expanded_cluegiver = cluegiver_data.loc[:, 'base_text'].str.split('black: ', expand=True)[1].str.split(', tan: ', expand=True)
cluegiver_data.insert(1, 'assassins', expanded_cluegiver[0])
expanded_cluegiver = expanded_cluegiver[1].str.split(', targets: ', expand=True)
cluegiver_data.insert(2, 'innocent', expanded_cluegiver[0])
cluegiver_data.insert(3, 'targets', expanded_cluegiver[1])
cluegiver_data = cluegiver_data.drop('base_text', axis=1)
cluegiver_data.rename(columns={'output':'clue'}, inplace = True)
cluegiver_data.insert(3, 'num_targets', cluegiver_data['targets'].str.count(',')+1)
cluegiver_data

,assassins,innocent,targets,num_targets,clue
0,"['switch', 'charge', 'strike']","['fall', 'march', 'scientist', 'link', 'drop',...",['sub'],1,sandwich
1,"['switch', 'charge', 'strike']","['fall', 'march', 'scientist', 'link', 'drop',...",['disease'],1,virus
2,"['switch', 'charge', 'strike']","['fall', 'march', 'scientist', 'link', 'drop',...",['sound'],1,hear
3,"['switch', 'charge', 'strike']","['fall', 'march', 'scientist', 'link', 'drop',...",['field'],1,football
4,"['switch', 'charge', 'strike']","['fall', 'march', 'scientist', 'link', 'drop',...",['roulette'],1,russian
...,...,...,...,...,...
7698,"['novel', 'racket', 'spring']","['march', 'embassy', 'contract', 'fall', 'row'...",['switch'],1,board
7699,"['novel', 'racket', 'spring']","['march', 'embassy', 'contract', 'fall', 'row'...",['day'],1,sun
7700,"['novel', 'racket', 'spring']","['march', 'embassy', 'contract', 'fall', 'row'...",['crash'],1,air
7701,"['novel', 'racket', 'spring']","['march', 'embassy', 'contract', 'fall', 'row'...",['plot'],1,map


In [3]:
# data downloaded from the 'generate guess task'
import numpy as np
guesser_data = pandas.read_csv('./baseline_data/guesser_shaikh.csv', index_col = 0, delimiter=';')

expanded_guesser = guesser_data.loc[:, 'base_text'].str.split('remaining: ', expand=True)[1].str.split(', hint: ', expand=True)
guesser_data.insert(1, 'board', expanded_guesser[0])
guesser_data.insert(2, 'hint', expanded_guesser[1])
guesser_data = guesser_data.drop('base_text', axis=1)

guesser_data.rename(columns={'output':'guesses'}, inplace = True)
guesser_data.insert(1, 'num_board', guesser_data['board'].str.count(',')+1)
guesser_data.insert(3, 'num_guesses', guesser_data['guesses'].str.count(',')+1)
guesser_data

,board,num_board,hint,num_guesses,guesses
0,"['spot', 'charge', 'novel', 'sound', 'march', ...",25,sandwich,1.0,green
1,"['spot', 'charge', 'novel', 'sound', 'march', ...",24,virus,1.0,disease
2,"['spot', 'charge', 'novel', 'sound', 'march', ...",23,hear,1.0,sound
3,"['spot', 'charge', 'novel', 'march', 'scientis...",22,football,1.0,field
4,"['spot', 'charge', 'novel', 'march', 'scientis...",21,russian,1.0,novel
...,...,...,...,...,...
7698,"['crash', 'novel', 'comic', 'march', 'racket',...",22,board,1.0,switch
7699,"['crash', 'novel', 'comic', 'march', 'racket',...",21,sun,1.0,day
7700,"['crash', 'novel', 'comic', 'march', 'racket',...",20,air,1.0,crash
7701,"['plot', 'draft', 'link', 'spring', 'wake', 'n...",19,map,1.0,plot


I assume that both dataframes are sorted in the same way and that I can recreate the game interactions. To check whether this assumption is true, I compare the 'clue' column from the clue-generation-task with the 'hint' column in the generate-guess-task:

In [4]:

import math
full_data = cluegiver_data.join(guesser_data)

full_data.dropna(inplace = True)
full_data['num_guesses'] = full_data['num_guesses'].astype(int)
assert (full_data['clue'] == full_data['hint']).all(), 'Columns clue and hint do not match!'

AssertionError: Columns clue and hint do not match!

The colums do not match, let's see where they do not:

In [5]:
full_data[full_data['clue'] != full_data['hint']]

,assassins,innocent,targets,num_targets,clue,board,num_board,hint,num_guesses,guesses
1299,"['cycle', 'scientist', 'strike']","['conductor', 'genius', 'charge', 'spot', 'cod...",['agent'],1,7,"['force', 'death', 'spot', 'charge', 'code', '...",24,007,1,agent
1500,"['centaur', 'pass', 'charge']","['leprechaun', 'mass', 'capital', 'light', 'so...",['spy'],1,7,"['death', 'center', 'row', 'centaur', 'spy', '...",17,007,2,"spy, charge"
1800,"['check', 'spring', 'field']","['plot', 'mount', 'degree', 'day', 'boom', 'da...","['spy', 'agent', 'novel']",3,7,"['crash', 'death', 'match', 'novel', 'comic', ...",25,007,1,bond
1801,"['check', 'spring', 'field']","['plot', 'mount', 'degree', 'day', 'boom', 'da...","['agent', 'spy', 'novel']",3,7,"['crash', 'death', 'match', 'novel', 'comic', ...",24,007,2,"spy, agent"
1916,"['light', 'part', 'check']","['cycle', 'comic', 'scientist', 'luck', 'centa...",['bond'],1,7,"['trip', 'slip', 'match', 'code', 'comic', 'sc...",23,007,1,agent
1918,"['light', 'part', 'check']","['cycle', 'comic', 'scientist', 'luck', 'centa...",['bond'],1,7,"['slip', 'match', 'code', 'comic', 'scientist'...",21,007,3,"state, revolution, bond"
1954,"['time', 'figure', 'fall']","['air', 'spot', 'strike', 'boom', 'trip', 'mam...",['spy'],1,7,"['trip', 'war', 'spot', 'slip', 'center', 'mat...",25,007,1,spy
2937,"['cast', 'capital', 'unicorn']","['contract', 'compound', 'field', 'millionaire...",['fair'],1,TRUE,"['war', 'sound', 'millionaire', 'strike', 'clu...",19,true,1,fair
3108,"['poison', 'state', 'cover']","['club', 'plot', 'pass', 'opera', 'light', 'so...",['spy'],1,7,"['lead', 'spot', 'light', 'soul', 'state', 'po...",25,007,1,spy
4816,"['vacuum', 'state', 'wind']","['force', 'boom', 'space', 'ray', 'check', 'la...","['spy', 'agent']",2,7,"['force', 'wind', 'charge', 'novel', 'drop', '...",25,007,1,agent


The mismatch only occurs for numbers ('007' was converted to 7) and booleans (true is capitalised). I believe this to be an issue on my side, because I deleted the demographic information by hand and my program probably changed those values based on assumptions that 007 should be an integer that does not have leading zeros. But as this dataframe is exhaustive, this is not an issue. Assuming that the 'hint' column retains the original information, I will drop the 'clue' column instead.

Assumptions:
- Sequential rows seem to occur in blocks of having the same assassins. I assume that these rows all belong to the same game and the same player and that they were not sorted by anything, but where actually played in this order.
- Two of these sequential blocks with the same assassins seem to have the same overall board. I assume that these two blocks form the two player sides in a game. I would have assumed that they play in an alternating fashion, but this cannot be proven by the data. For that, the remaining board for the guesser should have been updated with the guesses from the other player as well, but the data only suggests that the board is updated with the guessers own guesses for him, which does not abide by the games rules. Assuming, they made a mistake by writing the remaining board down, I will continue to assume that the players played alternatingly.


The following code introduces these assumptions in form of writing down game and turn numbers for each row. For each game, there is also counted how many team words, innocent words, target words (that were targeted by the cluegiver), and assassins were uncovered. Note that I do not accumulate this information by alternating between the players but by only going through them in one by one, the same as they are ordered in the dataset. This way, only the last row actually contains the complete game state.

In [6]:
full_data.drop('clue', axis =1, inplace = True)
full_data.insert(0, 'game', 0)
full_data.insert(1, 'turn', 0)
game = 0
turn = 0
game_col = []
turn_col = []
game_ass = ""
game_inno = ""
assassins_revealed_col = []
inno_revealed_col = []
targets_revealed_col = []
team_revealed_col = []
for index, row in full_data.iterrows():
    if row['num_board'] == 25 and game_ass != row['assassins'] and game_inno != row['innocent']:
        # new game
        game += 0.5
        turn = 1 if int(game) != float(game) else 2
        game_ass = row['assassins']
        game_inno = row['innocent']
        if int(game) != float(game):
            assassins_revealed = 0
            inno_revealed = 0
            targets_revealed = 0
            team_revealed = 0
        
    guesses = row['guesses'].split(', ')
    for guess in guesses:
        if guess in eval(row['assassins']):
            assassins_revealed += 1
        elif guess in eval(row['innocent']):
            inno_revealed += 1
        elif guess in eval(row['targets']):
            targets_revealed += 1
        else:
            team_revealed += 1
    game_col.append(math.floor(game + 0.5))
    turn_col.append(turn)
    turn += 2
    assassins_revealed_col.append(assassins_revealed)
    inno_revealed_col.append(inno_revealed)
    targets_revealed_col.append(targets_revealed)
    team_revealed_col.append(team_revealed)

full_data['game'] = game_col
full_data['turn'] = turn_col
full_data['assassins revealed'] = assassins_revealed_col
full_data['innocents revealed'] = inno_revealed_col
full_data['targets revealed'] = targets_revealed_col
full_data['team revealed'] = team_revealed_col
full_data[0:20]

,game,turn,assassins,innocent,targets,num_targets,board,num_board,hint,num_guesses,guesses,assassins revealed,innocents revealed,targets revealed,team revealed
0,1,1,"['switch', 'charge', 'strike']","['fall', 'march', 'scientist', 'link', 'drop',...",['sub'],1,"['spot', 'charge', 'novel', 'sound', 'march', ...",25,sandwich,1,green,0,0,0,1
1,1,3,"['switch', 'charge', 'strike']","['fall', 'march', 'scientist', 'link', 'drop',...",['disease'],1,"['spot', 'charge', 'novel', 'sound', 'march', ...",24,virus,1,disease,0,0,1,1
2,1,5,"['switch', 'charge', 'strike']","['fall', 'march', 'scientist', 'link', 'drop',...",['sound'],1,"['spot', 'charge', 'novel', 'sound', 'march', ...",23,hear,1,sound,0,0,2,1
3,1,7,"['switch', 'charge', 'strike']","['fall', 'march', 'scientist', 'link', 'drop',...",['field'],1,"['spot', 'charge', 'novel', 'march', 'scientis...",22,football,1,field,0,0,3,1
4,1,9,"['switch', 'charge', 'strike']","['fall', 'march', 'scientist', 'link', 'drop',...",['roulette'],1,"['spot', 'charge', 'novel', 'march', 'scientis...",21,russian,1,novel,0,1,3,1
5,1,11,"['switch', 'charge', 'strike']","['fall', 'march', 'scientist', 'link', 'drop',...",['sub'],1,"['spot', 'charge', 'march', 'scientist', 'drop...",20,sandwich,1,spot,0,2,3,1
6,1,13,"['switch', 'charge', 'strike']","['fall', 'march', 'scientist', 'link', 'drop',...",['roulette'],1,"['soul', 'check', 'strike', 'link', 'roulette'...",19,game,1,strike,1,2,3,1
7,1,2,"['fall', 'switch', 'roulette']","['march', 'link', 'sub', 'drop', 'disease', 'c...",['boom'],1,"['spot', 'charge', 'novel', 'sound', 'march', ...",25,bomb,1,boom,1,2,4,1
8,1,4,"['fall', 'switch', 'roulette']","['march', 'link', 'sub', 'drop', 'disease', 'c...",['tag'],1,"['spot', 'charge', 'novel', 'sound', 'march', ...",24,name,1,tag,1,2,5,1
9,1,6,"['fall', 'switch', 'roulette']","['march', 'link', 'sub', 'drop', 'disease', 'c...",['charge'],1,"['spot', 'charge', 'novel', 'sound', 'march', ...",23,electricity,1,charge,1,2,6,1


Next, I will try to find out how many turns each game took:

In [9]:
full_data['num_board'].value_counts()

25    1588
23    1285
21    1155
22    1142
24    1065
20     549
19     404
18     256
17     114
16      41
15      13
14       9
13       4
Name: num_board, dtype: int64

In [10]:
list(full_data['turn'].value_counts())

[794,
 794,
 764,
 753,
 716,
 713,
 663,
 660,
 588,
 583,
 143,
 131,
 87,
 85,
 54,
 40,
 18,
 16,
 7,
 6,
 3,
 2,
 2,
 2,
 1]

In [11]:
end_turn = list(full_data['turn'].value_counts())
for i in range(0, len(end_turn) - 1):
    end_turn[i] -= end_turn[i+1]
end_turn

[0,
 30,
 11,
 37,
 3,
 50,
 3,
 72,
 5,
 440,
 12,
 44,
 2,
 31,
 14,
 22,
 2,
 9,
 1,
 3,
 1,
 0,
 0,
 1,
 1]

In [12]:
avg = 0
for i in range(len(end_turn)):
    avg += end_turn[i] * (i+1)
avg/sum(end_turn)

9.603274559193954

In [273]:
sum(end_turn)

794

In [268]:
len(full_data)/sum(end_turn)

9.603274559193954

My average number of turns comes out at 9.6 instead of the reported 9.7, but I did exclude some turns where after merging cluegiver and guesser, there was only the clue but no guess, which is not a completed turn.

In [186]:
sum(end_turn[0:8]), sum(end_turn[8:])

(206, 588)

Going by the original Codenames Duet rules, only games that took 9 turns or less could have been won (if also all team cards and no assassins were revealed), so 588 games are definitely lost by this rule.

In [13]:
full_data

,game,turn,assassins,innocent,targets,num_targets,board,num_board,hint,num_guesses,guesses,assassins revealed,innocents revealed,targets revealed,team revealed
0,1,1,"['switch', 'charge', 'strike']","['fall', 'march', 'scientist', 'link', 'drop',...",['sub'],1,"['spot', 'charge', 'novel', 'sound', 'march', ...",25,sandwich,1,green,0,0,0,1
1,1,3,"['switch', 'charge', 'strike']","['fall', 'march', 'scientist', 'link', 'drop',...",['disease'],1,"['spot', 'charge', 'novel', 'sound', 'march', ...",24,virus,1,disease,0,0,1,1
2,1,5,"['switch', 'charge', 'strike']","['fall', 'march', 'scientist', 'link', 'drop',...",['sound'],1,"['spot', 'charge', 'novel', 'sound', 'march', ...",23,hear,1,sound,0,0,2,1
3,1,7,"['switch', 'charge', 'strike']","['fall', 'march', 'scientist', 'link', 'drop',...",['field'],1,"['spot', 'charge', 'novel', 'march', 'scientis...",22,football,1,field,0,0,3,1
4,1,9,"['switch', 'charge', 'strike']","['fall', 'march', 'scientist', 'link', 'drop',...",['roulette'],1,"['spot', 'charge', 'novel', 'march', 'scientis...",21,russian,1,novel,0,1,3,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7698,794,8,"['novel', 'racket', 'spring']","['march', 'embassy', 'contract', 'fall', 'row'...",['switch'],1,"['crash', 'novel', 'comic', 'march', 'racket',...",22,board,1,switch,0,0,11,0
7699,794,10,"['novel', 'racket', 'spring']","['march', 'embassy', 'contract', 'fall', 'row'...",['day'],1,"['crash', 'novel', 'comic', 'march', 'racket',...",21,sun,1,day,0,0,12,0
7700,794,12,"['novel', 'racket', 'spring']","['march', 'embassy', 'contract', 'fall', 'row'...",['crash'],1,"['crash', 'novel', 'comic', 'march', 'racket',...",20,air,1,crash,0,0,13,0
7701,794,14,"['novel', 'racket', 'spring']","['march', 'embassy', 'contract', 'fall', 'row'...",['plot'],1,"['plot', 'draft', 'link', 'spring', 'wake', 'n...",19,map,1,plot,0,0,14,0


Aggregating statistics for only the games, taking the accumulated result of which cards were revealed during the game to calculate which games are won.

In [14]:
game_statistics = full_data.groupby('game').agg({'turn': 'max', 
                                                 'num_board':'min', 
                                                 'assassins revealed': 'last', 
                                                 'innocents revealed': 'last', 
                                                 'targets revealed': 'last', 
                                                 'team revealed': 'last'})

game_statistics

,turn,num_board,assassins revealed,innocents revealed,targets revealed,team revealed
game,,,,,,
1,14,19,1,2,10,1
2,16,15,0,4,15,0
3,15,18,0,0,15,0
4,15,18,0,0,15,0
5,15,18,0,0,15,0
...,...,...,...,...,...,...
790,13,18,0,0,15,0
791,21,15,2,12,5,9
792,4,22,1,1,4,0


Introducing 'win' and 'lose' columns to all games, for now ignoring the above information with the maximum number of turns:

In [15]:
game_statistics['revealed all targets'] = game_statistics['targets revealed'] >= 15
game_statistics['revealed all team'] = game_statistics['targets revealed'] + game_statistics['team revealed'] >= 15
game_statistics['lost through assassin'] = game_statistics['assassins revealed'] >= 1
game_statistics

,turn,num_board,assassins revealed,innocents revealed,targets revealed,team revealed,revealed all targets,revealed all team,lost through assassin
game,,,,,,,,,
1,14,19,1,2,10,1,False,False,True
2,16,15,0,4,15,0,True,True,False
3,15,18,0,0,15,0,True,True,False
4,15,18,0,0,15,0,True,True,False
5,15,18,0,0,15,0,True,True,False
...,...,...,...,...,...,...,...,...,...
790,13,18,0,0,15,0,True,True,False
791,21,15,2,12,5,9,False,False,True
792,4,22,1,1,4,0,False,False,True


In [16]:
# How many games revealed all targeted words (not as important as actually winning the game, but a good precision metric)
game_statistics['revealed all targets'].value_counts()

False    736
True      58
Name: revealed all targets, dtype: int64

In [17]:
# games lost through the assassin
game_statistics['lost through assassin'].value_counts()

False    582
True     212
Name: lost through assassin, dtype: int64

In [18]:
# games won (if ignoring number of turns)
game_statistics['revealed all team'].value_counts()

False    638
True     156
Name: revealed all team, dtype: int64

So far, I cannot reproduce numbers that are stated in the paper (199 wins, 595 losses), but with the additional rule of only having maximum of 9 turns, this win number will get even smaller.

Looking at games that were not lost through the assassin and not won by revealing all team cards:

In [19]:
# seemingly unfinished games (if ignoring max 9 turns)
game_statistics[game_statistics['revealed all team'] == game_statistics['lost through assassin']]

,turn,num_board,assassins revealed,innocents revealed,targets revealed,team revealed,revealed all targets,revealed all team,lost through assassin
game,,,,,,,,,
61,11,20,0,0,11,0,False,False,False
66,7,22,0,0,8,0,False,False,False
81,12,20,0,0,11,0,False,False,False
82,10,20,0,2,8,1,False,False,False
83,10,21,0,0,10,0,False,False,False
...,...,...,...,...,...,...,...,...,...
783,10,21,0,0,10,0,False,False,False
784,10,21,0,0,10,0,False,False,False
786,10,21,0,0,10,0,False,False,False


In [20]:
# How many target cards were revealed in these unfinished games
game_statistics[game_statistics['revealed all team'] == game_statistics['lost through assassin']]['targets revealed'].value_counts()

10    234
11     62
9      40
8      27
12     20
13     14
7      14
14      6
5       3
4       3
6       3
Name: targets revealed, dtype: int64

In [21]:
# How many team cards were revealed in these unfinished games
game_statistics[game_statistics['revealed all team'] == game_statistics['lost through assassin']]['team revealed'].value_counts()

0    315
1     64
2     22
3     21
4      3
5      1
Name: team revealed, dtype: int64

In [22]:
# looking at an example game that was neither won nor lost
full_data[full_data['game'] == 82]

,game,turn,assassins,innocent,targets,num_targets,board,num_board,hint,num_guesses,guesses,assassins revealed,innocents revealed,targets revealed,team revealed
948,82,1,"['wind', 'life', 'trip']","['war', 'force', 'sub', 'degree', 'embassy', '...",['field'],1,"['force', 'trip', 'war', 'center', 'wind', 'ch...",25,baseball,1,field,0,0,1,0
949,82,3,"['wind', 'life', 'trip']","['war', 'force', 'sub', 'degree', 'embassy', '...",['charge'],1,"['force', 'trip', 'war', 'center', 'wind', 'ch...",24,battery,1,charge,0,0,2,0
950,82,5,"['wind', 'life', 'trip']","['war', 'force', 'sub', 'degree', 'embassy', '...",['capital'],1,"['force', 'trip', 'war', 'center', 'wind', 'co...",23,letter,1,capital,0,0,3,0
951,82,7,"['wind', 'life', 'trip']","['war', 'force', 'sub', 'degree', 'embassy', '...",['mass'],1,"['force', 'trip', 'war', 'center', 'wind', 'co...",22,priest,1,mass,0,0,4,0
952,82,9,"['wind', 'life', 'trip']","['war', 'force', 'sub', 'degree', 'embassy', '...",['comic'],1,"['force', 'trip', 'war', 'center', 'wind', 'co...",21,funny,1,comic,0,0,5,0
953,82,2,"['comic', 'ray', 'trip']","['force', 'sub', 'point', 'degree', 'embassy',...","['ghost', 'witch']",2,"['force', 'trip', 'war', 'center', 'wind', 'ch...",25,halloween,2,"ghost, witch",0,0,7,0
954,82,4,"['comic', 'ray', 'trip']","['force', 'sub', 'point', 'degree', 'embassy',...","['center', 'row']",2,"['force', 'trip', 'war', 'center', 'wind', 'ch...",23,middle,1,center,0,0,8,0
955,82,6,"['comic', 'ray', 'trip']","['force', 'sub', 'point', 'degree', 'embassy',...","['wind', 'light']",2,"['force', 'trip', 'war', 'wind', 'charge', 'co...",22,elements,1,mass,0,1,8,0
956,82,8,"['comic', 'ray', 'trip']","['force', 'sub', 'point', 'degree', 'embassy',...","['wind', 'light']",2,"['force', 'trip', 'war', 'wind', 'charge', 'co...",21,element,1,force,0,2,8,0
957,82,10,"['comic', 'ray', 'trip']","['sub', 'point', 'degree', 'embassy', 'charge'...","['wind', 'light']",2,"['trip', 'war', 'wind', 'charge', 'comic', 'li...",20,element,1,part,0,2,8,1


In [23]:
game_statistics[game_statistics['revealed all team'] == True]['turn'].value_counts().sort_index()

4      1
5      1
6      2
7      4
8     14
9     13
10    25
11     6
12    10
13     8
14    14
15    12
16    27
17     4
18     8
19     1
20     4
21     1
26     1
Name: turn, dtype: int64

# Analysis

In [24]:
# Average number of targets, slightly lower than original 1.24, but rounded up it fits
full_data['num_targets'].mean()

1.2375081967213115

In [25]:
# Average number of guesses
full_data['num_guesses'].mean()

1.2304262295081967

In [26]:
# 212 games were lost through uncovering an assassin
game_statistics['lost through assassin'].value_counts()

False    582
True     212
Name: lost through assassin, dtype: int64

In [27]:
# 156 games were 'won' by revealing all team words
game_statistics['revealed all team'].value_counts()

False    638
True     156
Name: revealed all team, dtype: int64

In [28]:
game_statistics[game_statistics['revealed all team'] == True]['turn'].value_counts().sort_index()

4      1
5      1
6      2
7      4
8     14
9     13
10    25
11     6
12    10
13     8
14    14
15    12
16    27
17     4
18     8
19     1
20     4
21     1
26     1
Name: turn, dtype: int64

In [29]:
# now looking again at the maximum number of turns
valid_games = game_statistics[game_statistics['turn'] <= 9]
valid_games

,turn,num_board,assassins revealed,innocents revealed,targets revealed,team revealed,revealed all targets,revealed all team,lost through assassin
game,,,,,,,,,
8,4,24,1,1,2,1,False,False,True
9,6,21,1,2,6,0,False,False,True
17,4,23,1,2,5,0,False,False,True
21,5,20,1,0,8,1,False,False,True
23,6,21,1,2,10,0,False,False,True
...,...,...,...,...,...,...,...,...,...
749,8,18,0,0,15,0,True,True,False
768,7,19,0,0,7,8,False,True,False
770,2,25,1,1,1,0,False,False,True


In [30]:
# valid games (that are not lost by taking over 9 turns) that are lost through uncovering an assassin
valid_games['lost through assassin'].value_counts()

True     166
False     36
Name: lost through assassin, dtype: int64

In [31]:
# valid games that are actually won within 9 turns
valid_games['revealed all team'].value_counts()

False    167
True      35
Name: revealed all team, dtype: int64

In [33]:
valid_games[valid_games['lost through assassin'] == valid_games['revealed all team']]

,turn,num_board,assassins revealed,innocents revealed,targets revealed,team revealed,revealed all targets,revealed all team,lost through assassin
game,,,,,,,,,
66,7,22,0,0,8,0,False,False,False


One game of the valid 202 games still is unfinished within 9 turns, only 35 are actually won and the remaining 166 are lost through the assassin. This means for all recorded 794 games:
- 35 wins
- 166 losses through assassins
- 592 losses through exceeding 9 turns
- 1 unfinished game

This leaves us with 793 usable games, of which only 35 were won, resulting in a 4.4% success rate for humans. I would strongly argue that this is a very bad baseline, from my own experience, people have less difficulties playing the game and should win more often. I would assume that either the experimenters or participants did not understand the 9-turn rule, as it is not mentioned in the paper and apparently not upheld in the data as well, but even ignoring this rule, I cannot reproduce the data reported in the paper. I sent a clarification request to Omar Shaikh.